# Geo-Spatial Visualisation


## Introduction

In this chapter, you'll learn the basics of geospatial visualisation using code. If you're new to geospatial analysis, you should look at the introduction page first.

You should be aware when following this chapter that installing geographic analysis packages isn't always the easiest and things can and do go wrong! (Some geospatial analysis courses recommend running everything in a Docker container.)

### Imports and packages

We'll be using [**geopandas**](https://geopandas.org/index.html), the go-to package for vector spatial analysis in Python. The easiest way to install this package is using `conda install geopandas`; if you want to install it via pip then look at the [install instructions](https://geopandas.org/install.html). 

Let's import some of the packages we'll be using:

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import os
import numpy as np
import geopandas as gpd

In [ ]:
# TODO hide cell
# Set max rows displayed for readability
pd.set_option('display.max_rows', 6)
# Plot settings
plt.style.use('plot_style.txt')
# For this page, use data limits and bigger default fig size
plt.style.use({'axes.autolimit_mode': 'data', 'figure.figsize': (10, 8)})

## Maps

If you've looked at the introductory page on geospatial analysis, you'll know it's easy to make basic maps: you just need to load a shapefile and use the `.plot` method.

In [ ]:
df = gpd.read_file(os.path.join('data', 'geo', 'uk_lad', 'Local_Authority_Districts__May_2020__UK_BUC.shp'))
df.plot();

As it goes, this is not very attractive, so let's see some options for customisation that will make it a little better. It's rare that you'll want to include the axes on maps, and these can be turned off by turning everything to do with the axes off. There are two ways to do further manipulations of the figure axis: calling plot returns an axis object or we can create one and then pass `ax=ax_name` to plot as a keyword argument. Colour can be changed using the `color=` keyword.

In [ ]:
ax = df.plot(color='#2ca25f')
ax.axis('off');

The lines that divide up the different local authority districts are faint. They can be controlled with the `edgecolor` and `linewidth` keywords. We can also change the background using the `fig.patch.set_facecolor` method, and add a scale using an extra package, [**matplotlib-scalebar**](https://github.com/ppinard/matplotlib-scalebar).

In [ ]:
from matplotlib_scalebar.scalebar import ScaleBar

fig, ax = plt.subplots()
df.plot(color='#2ca25f', edgecolor='k', linewidth=0.2,
        facecolor='blue',
        ax=ax)
ax.axis('off')
fig.patch.set_facecolor('#9ecae1')
# Create scale bar
scalebar = ScaleBar(1, box_alpha=0,
                    location='lower right', length_fraction=0.25,
                    font_properties={'size':12})
ax.add_artist(scalebar)
plt.show()

## Choropleths

A choropleth map shows different areas in colours according to a statistic that represents an aggregate summary of a geographic characteristic within each area. Population density or per-capita income are good examples of characteristics. The statistic shown might be unique values, equal intervals, quantiles, or the Fisher-Jenks natural breaks.

First, though, let's create a basic choropleth.

In [ ]:
pay = pd.read_csv('/Users/arthurturrell/Desktop/ashe_lad_median_pay_2020.csv')
pay = pay.rename(columns={'lad': 'LAD20CD'})

In [ ]:
df = df.merge(pay, on=['LAD20CD'], how='inner')

In [ ]:
col = 'Log median weekly pay (2020 GBP)'
df[col] = np.log(df['Median weekly pay (2020 GBP)'])

fig, ax = plt.subplots()
ax.set_title(col, loc='left')
df.plot(ax=ax, column=col, legend=True,
        legend_kwds={'label': '',
                     'shrink': 0.6},
        vmin=round(df[col].min()),
        vmax=round(df[col].max()),
        )
ax.axis('off')
plt.tight_layout()
plt.show();

This used **geopandas**. There's a dedicated plotting tool called [**geoplot**]() as well.

In [ ]:
import geoplot as gplt
import geoplot.crs as gcrs

gplt.choropleth(df.to_crs('EPSG:4326'), hue=col,
                projection=gcrs.AlbersEqualArea(),
                cmap='viridis', legend=True);

Another way to create choropleths is to split the variable into a distinct number of ranges according to a scheme. In the below, we use `scheme='Quantiles'` with `k=4` to produce a choropleth with four distinct groups.

In [ ]:
fig, ax = plt.subplots(figsize=(8, 10))
ax.set_title(col, loc='left')
ax.axis('off')
df.plot(ax=ax, column=col, legend=True,
        scheme='Quantiles', k=4,
        legend_kwds={'loc': 2});

A third kind of choropleth has distinct levels based on pre-existing categories. Our data doesn't have any of those, so let's generate some just to show how it works.

https://darribas.org/gds_course/content/bD/lab_D.html

In [ ]:
df['cat_col'] = df['LAD20CD'].apply(lambda x: x[0])
df.iloc[:5, -3:]

In [ ]:
fig, ax = plt.subplots()
df.plot(column='cat_col', categorical=True,
        ax=ax, legend=True,
        legend_kwds={'loc': 1, 'frameon': True})
ax.set_axis_off()
plt.show()

This is useful for plotting streets of different types.

## Cartogram

In [ ]:
df_wales = df[df['LAD20CD'].str.contains('W')].to_crs('EPSG:4326').fillna(0.)

ax = gplt.cartogram(
    df_wales, scale='Median weekly pay (2020 GBP)', projection=gcrs.AlbersEqualArea(),
)
gplt.polyplot(df_wales, facecolor='lightgray', edgecolor='white', linewidth=0.5, ax=ax)

## Quadtree

## KDE plot